In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sd3_pipeline import SD3CNPipeline
from sd3 import SD3CNModel
from diffusers import AutoencoderKL, FlowMatchEulerDiscreteScheduler
import torch

2024-11-13 21:18:07.788236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-13 21:18:07.788296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-13 21:18:07.789658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 21:18:07.796189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 21:18:08.537976: W tensorflow/compiler/tf2

In [4]:
from text_embed import encode_prompt, get_precomputed_tensors

In [5]:
device = 'cuda:0'

In [6]:
transformer = SD3CNModel.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers", 
    subfolder="transformer", 
    # torch_dtype=torch.float16,
).to(device)

In [7]:
vae = AutoencoderKL.from_pretrained(
            "stabilityai/stable-diffusion-3-medium",
            subfolder="vae",
            revision="refs/pr/26").to(device)

In [8]:
scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers", subfolder="scheduler"
)

In [9]:
tensor_list = get_precomputed_tensors(device=device)
        
pixel_list = [x['img'] for x in tensor_list]
hint_list = [x['hint'] for x in tensor_list]
prompt_embed_list = [x['prompt_embeds'] for x in tensor_list]
pooled_prompt_embed_list = [x['pooled_prompt_embeds'] for x in tensor_list]

index = 0
prompt_embeds = prompt_embed_list[index].to(device)
pooled_prompt_embeds = pooled_prompt_embed_list[index].to(device)
print(tensor_list[index]['prompt'])



pale golden rod circle with old lace background


In [10]:
pipe = SD3CNPipeline(transformer, scheduler, vae, device)

In [11]:
res = pipe(prompt_embeds=prompt_embeds,
           pooled_prompt_embeds=pooled_prompt_embeds,
           # control_hidden_states=control_hidden_states,
           # index_block_location=index_block_location,
           guidance_scale=0.5)
image = res.images[0]

> /home/sj/workspace/diffusion/sd3_pipeline.py(268)__call__()
    266 
    267         import pdb; pdb.set_trace()
--> 268         height = height or self.default_sample_size * self.vae_scale_factor
    269         width = width or self.default_sample_size * self.vae_scale_factor
    270 



ipdb>  height
ipdb>  n


> /home/sj/workspace/diffusion/sd3_pipeline.py(269)__call__()
    267         import pdb; pdb.set_trace()
    268         height = height or self.default_sample_size * self.vae_scale_factor
--> 269         width = width or self.default_sample_size * self.vae_scale_factor
    270 
    271         # 1. Check inputs. Raise error if not correct



ipdb>  n


> /home/sj/workspace/diffusion/sd3_pipeline.py(273)__call__()
    271         # 1. Check inputs. Raise error if not correct
    272 
--> 273         self._guidance_scale = guidance_scale
    274         self._clip_skip = clip_skip
    275         self._joint_attention_kwargs = joint_attention_kwargs



ipdb>  q


In [23]:
import numpy as np

In [26]:
np.asarray(image).shape

(1024, 1024, 3)

In [21]:
image.shape

AttributeError: 'Image' object has no attribute 'shape'